In [ ]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
# Charger un tableau NumPy à partir du fichier HDF5
with h5py.File('data_0_comp.h5', 'r') as hf:
    X_train_img_id = hf['image_ids'][:]
    X_train_product_id = hf['product_ids'][:]
    X_train_hog = hf['X_train_hog'][:]
    X_train_hsv = hf['X_train_hsv'][:]
    

In [26]:
df_X_train_txt = pd.read_csv('text_lemm.csv', delimiter=',')

In [58]:
#on filtre sur les images qui ont bien été traitées
df_features = df_X_train_txt[df_X_train_txt['imageid'].isin(X_train_img_id)]
y = df_features['prdtypecode']

In [67]:
#Initialisation of the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
#Adding .values.astype('U') to avoid error np.nan is an invalid document, expected byte or unicode string.
X_train_tfidf = vectorizer.fit_transform(df_X_train_txt['Text'].values.astype('U'))


In [69]:
# Séparer la variable explicative de la variable à prédire
X_tfidf, y_tfidf = df_X_train_txt['Text'], y

# Séparer le jeu de données en données d'entraînement et données test 
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_tfidf, test_size=0.2, random_state = 30)

#Importer le package du TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer un vectorisateur 
vec_tfidf = TfidfVectorizer()

# Mettre à jour la valeur de X_train_tfidf et X_test_tfidf
X_train_tfidf = vec_tfidf.fit_transform(X_train_tfidf)
X_test_tfidf = vec_tfidf.transform(X_test_tfidf)


In [ ]:
# Créer un classificateur clf_tfidf et entraîner le modèle sur l'ensemble d'entraînement
clf_tfidf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_tfidf, y_train_tfidf)

# Calculer les prédictions 
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)


In [ ]:
# Calcul et affichage de classification_report
print( classification_report(y_test_tfidf, y_pred_tfidf) )

# Calcul et affichage de la matrice de confusion
conf_matrix_tfidf = pd.crosstab(y_test_tfidf, y_pred_tfidf, rownames=['Classe réelle'], colnames=['Classe prédite'])
conf_matrix_tfidf
